In [1]:
import telepot
import openai
import nltk
#nltk.download('punkt')
#nltk.download('stopwords')
import pandas as pd
from nltk.tokenize import word_tokenize
from nltk.stem.snowball import SnowballStemmer
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score

In [2]:
fake = pd.read_csv('Fake.csv')
true = pd.read_csv('True.csv')

fake['target'] = 0
true['target'] = 1

data = pd.concat([fake,true], axis=0)
date = data.reset_index(drop=True)

data=data.drop(['subject', 'date', 'title'], axis=1)

In [3]:
stemmer = SnowballStemmer('english')
stop_words = stopwords.words('english')

In [4]:
def preprocess_text(text):
    tokens = word_tokenize(text)
    stemmed_tokens = [stemmer.stem(token) for token in tokens]
    filtered_tokens = [token for token in stemmed_tokens if token not in stop_words]
    preprocessed_text = ' '.join(filtered_tokens)
    return preprocessed_text

In [5]:
data['text']=data['text'].apply(preprocess_text)

cv = CountVectorizer(max_features = 1500)
X = cv.fit_transform(data['text']).toarray()
y = data['target'].values

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [7]:
gnb = GaussianNB()
gnb.fit(X_train, y_train)
y_pred = gnb.predict(X_test)

In [8]:
acc = accuracy_score
print((acc(y_pred, y_test))*100)

88.94209354120267


In [9]:
TOKEN = '6634845571:AAHjdg54lWWxwom2uWo38aPHqBV09P6tovc'

In [10]:
def handle(msg):
    content_type, chat_type, chat_id = telepot.glance(msg)
    if content_type == 'text':
        msg=msg['text']
        preprocessed_text = preprocess_text(msg)

        X_new = cv.transform([preprocessed_text]).toarray()
        y_pred_new = gnb.predict(X_new)

        if y_pred_new[0] == 0:
            bot.sendMessage(chat_id, "Fake News")
        else:
            bot.sendMessage(chat_id, "True News")

In [11]:
if __name__ == '__main__':
    bot = telepot.Bot(TOKEN)
    bot.message_loop(handle)

    print("Listening for messages...")

    while True:
        pass


Listening for messages...
